# Soil Type Classification using Convolutional Neural Network

In [6]:
train_data_directory = 'new_image_dataset/train'
test_data_directory = 'new_image_dataset/test'

In [7]:
import cv2 # open cv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

load_model = tf.keras.models.load_model
image = tf.keras.preprocessing.image
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
Sequential = tf.keras.models.Sequential
Dense = tf.keras.layers.Dense
Activation = tf.keras.layers.Activation
Flatten = tf.keras.layers.Flatten
Conv2D = tf.keras.layers.Conv2D
MaxPooling2D = tf.keras.layers.MaxPooling2D

In [8]:
# initiate data preprocessing tools

# This step processes the images into a format that
# 1. makes the data readable to the model
# 2. provides more training material for the model to train from
# the `training_data_processor` below scales the data so that it can be
# a model input, but also takes each image and augments it so that
# the model can learn from multiple variations of the same image.
# it flips it horizontally, rotates it, shifts it, and more so that 
# the model learns from the soil photo rather than the orientation size
training_data_processor  = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    zoom_range = 0.2,
    rotation_range = 10,
    shear_range = 0.2,
    height_shift_range = 0.1,
    width_shift_range = 0.1,
)
# for the testing images, we don't need to create multiple variatinos
test_data_processor = ImageDataGenerator(rescale=1./255)


# load data into python

training_data = training_data_processor.flow_from_directory(
    train_data_directory,
    target_size = (256, 256), # pixels
    batch_size = 32,
    class_mode = 'categorical',
)

testing_data = test_data_processor.flow_from_directory(
    test_data_directory,
    target_size = (256, 256),
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = False
)

Found 565 images belonging to 8 classes.
Found 573 images belonging to 8 classes.


## Model building

In [9]:
# choose model parameters
num_conv_layers = 4
num_dense_layers = 3
layer_size = 29 # limit the layer size as we dont want to over fit the model
num_training_epochs = 50
MODEL_NAME = 'final_model'

CLASSES = ['alike','clay', 'gravel', 'humus', 'not', 'sand', 'silt',' yellow']

In [10]:
# initiate model variable
model = Sequential()

# begin adding properties to model variable
# e.g. add a convulutional layer
model.add(Conv2D(layer_size, (3,3), input_shape=(256, 256, 3)))
model.add(Activation('relu')) # rectified linear unit
model.add(MaxPooling2D(pool_size=(2,2)))

# add additional convolutional layers based on num_conv_layers
for _ in range(num_conv_layers-1):
    model.add(Conv2D(layer_size, (3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

# reduce dimensionality
model.add(Flatten())

# add fully connected "dense" layers if specified
for _ in range(num_dense_layers):
    model.add(Dense(layer_size))
    model.add(Activation('relu'))

# output layer
model.add(Dense(len(CLASSES)))
model.add(Activation('softmax'))

# compile the sequential model with all added properties
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

# use the data already loaded to train/tune the model
model_results = model.fit(
    training_data,
    epochs = num_training_epochs, 
    validation_data = testing_data)

# save the trained model
model.save(f'{MODEL_NAME}.h5')

Epoch 1/50
18/18 [==============================] - 29s 2s/step - loss: 1.8756 - accuracy: 0.2991 - val_loss: 1.7215 - val_accuracy: 0.3368
Epoch 2/50
18/18 [==============================] - 31s 2s/step - loss: 1.5866 - accuracy: 0.4407 - val_loss: 1.3922 - val_accuracy: 0.5131
Epoch 3/50
18/18 [==============================] - 29s 2s/step - loss: 1.3368 - accuracy: 0.5080 - val_loss: 1.1537 - val_accuracy: 0.5480
Epoch 4/50
11/18 [=================>............] - ETA: 9s - loss: 1.1200 - accuracy: 0.5767 

## Model Summary

In [ ]:
model.summary()

## Evaluation

In [ ]:
test_result = model.evaluate(testing_data)

print(test_result)

In [ ]:
import pandas as pd

training_dataframe = pd.DataFrame(model_results.history)
plt.plot(training_dataframe['accuracy'], label='Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
y_predicted = model.predict(testing_data)
y_predicted = [x.argmax() for x in y_predicted]

from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(testing_data.classes, y_predicted)

import seaborn as sb

sb.heatmap(matrix, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
def classify(image_fp):
    im = cv2.imread(image_fp) # load the image from the given file path (image_fp)
    plt.imshow(im[:,:,[2,1,0]]) # swap the colors because open-cv swaps the rgb colors
    img = image.load_img(image_fp, target_size = (256, 256))
    img = image.img_to_array(img)

    image_array = img / 255. # scale the image
    img_batch = np.expand_dims(image_array, axis = 0)

    class_ = CLASSES # possible output values
    from tensorflow import keras
    model = keras.models.load_model('final_model.h5')
    predicted_value = model.predict(img_batch)
    
    out  = f"""
     alike: {predicted_value[0][0]:.5f}
      clay: {predicted_value[0][1]:.5f}
    gravel: {predicted_value[0][2]:.5f}
     humus: {predicted_value[0][3]:.5f}
       not: {predicted_value[0][4]:.5f}
      peat: {predicted_value[0][5]:.5f}
      peat: {predicted_value[0][6]:.5f}
    yellow: {predicted_value[0][7]:.5f}

    Result: {(predicted_value[0][predicted_value.argmax()] * 100):.2f}% {class_[predicted_value.argmax()]}

    """
  # accuracy: {(accuracy[1]*100):.5f}
  #     loss: {(accuracy[0]*100):.5f}
    return out

In [ ]:
test_image_file_path = test_data_directory + '/alike/IMG_20221001_105510.jpg'
test_prediction = classify(test_image_file_path)

print(test_prediction)

## Convert final_model.h5 to TFLITE

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('final_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("final_model.tflite", "wb").write(tflite_model)

## Converting to tfjs

In [ ]:
import os
os.system("""
tensorflowjs_converter --input_format keras \
	final_model.h5 \
	tfjs""")